In [5]:
"""
# CNN Fully Convolutional Image Classification with TensorFlow
# https://www.learnopencv.com/cnn-fully-convolutional-image-classification-with-tensorflow/
# Anastasia Murzova
# JULY 13, 2020 1 COMMENT
"""
# https://www.tensorflow.org/api_docs/python/tf/keras/applications/
# Modules
# densenet module: DenseNet models for Keras.
# efficientnet module: EfficientNet models for Keras.
# imagenet_utils module: Utilities for ImageNet data preprocessing & prediction decoding.
# inception_resnet_v2 module: Inception-ResNet V2 model for Keras.
# inception_v3 module: Inception V3 model for Keras.
# mobilenet module: MobileNet v1 models for Keras.
# mobilenet_v2 module: MobileNet v2 models for Keras.
# nasnet module: NASNet-A models for Keras.
# resnet module: ResNet models for Keras.
# resnet50 module: Public API for tf.keras.applications.resnet50 namespace.
# resnet_v2 module: ResNet v2 models for Keras.
# vgg16 module: VGG16 model for Keras.
# vgg19 module: VGG19 model for Keras.
# xception module: Xception V1 model for Keras.
print(__doc__)


# CNN Fully Convolutional Image Classification with TensorFlow
# https://www.learnopencv.com/cnn-fully-convolutional-image-classification-with-tensorflow/
# Anastasia Murzova
# JULY 13, 2020 1 COMMENT



![](https://www.learnopencv.com/wp-content/uploads/2020/05/resNet18_resNet50_blocks.png)

In [1]:
# ResNet50 initial function
def ResNet50(include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=None,
             pooling=None,
             classes=1000,
             **kwargs):
  """Instantiates the ResNet50 architecture."""

  def stack_fn(x):
    x = stack1(x, 64, 3, stride1=1, name='conv2')
    x = stack1(x, 128, 4, name='conv3')
    x = stack1(x, 256, 6, name='conv4')
    return stack1(x, 512, 3, name='conv5')

  return ResNet(stack_fn, False, True, 'resnet50', include_top, 
      weights, input_tensor, input_shape, pooling, classes, **kwargs)

In [2]:
# TF ResNet basic pipeline: ResNet50 case
def ResNet(stack_fn,
           preact,
           use_bias,
           model_name='resnet',
           include_top=True,
           weights='imagenet',
           input_tensor=None,
           input_shape=None,
           pooling=None,
           classes=1000,
           classifier_activation='softmax',
           **kwargs):
    # ...
    x = layers.ZeroPadding2D(
        padding=((3, 3), (3, 3)),
        name='conv1_pad'
    )(img_input)
    x = layers.Conv2D(64, 7, strides=2, use_bias=use_bias, 
        name='conv1_conv')(x)

    x = layers.BatchNormalization(
        axis=bn_axis,
        epsilon=1.001e-5,
        name='conv1_bn'
    )(x)
    x = layers.Activation('relu', name='conv1_relu')(x)

    x = layers.ZeroPadding2D(
        padding=((1, 1), (1, 1)),
        name='pool1_pad'
    )(x)
    x = layers.MaxPooling2D(3, strides=2, name='pool1_pool')(x)

    # residual stacked block sequence
    x = stack_fn(x)

    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    imagenet_utils.validate_activation(
        classifier_activation,
        weights
    )
    x = layers.Dense(
        classes,
        activation=classifier_activation,
        name='predictions'
    )(x)
    # ...

In [ ]:
def fully_convolutional_resnet50(
    input_shape, num_classes=1000, pretrained_resnet=True, 
    use_bias=True,
):
    # init input layer
    img_input = Input(shape=input_shape)

    # define basic model pipeline
    x = ZeroPadding2D(padding=((3, 3), (3, 3)),
        name="conv1_pad")(img_input)
    x = Conv2D(64, 7, strides=2, use_bias=use_bias,
        name="conv1_conv")(x)
    x = BatchNormalization(axis=3, epsilon=1.001e-5,
        name="conv1_bn")(x)
    x = Activation("relu", name="conv1_relu")(x)

    x = ZeroPadding2D(padding=((1, 1), (1, 1)),
        name="pool1_pad")(x)
    x = MaxPooling2D(3, strides=2, name="pool1_pool")(x)

    # the sequence of stacked residual blocks
    x = stack1(x, 64, 3, stride1=1, name="conv2")
    x = stack1(x, 128, 4, name="conv3")
    x = stack1(x, 256, 6, name="conv4")
    x = stack1(x, 512, 3, name="conv5")

    # add avg pooling layer after feature extraction layers
    x = AveragePooling2D(pool_size=7)(x)

    # add final convolutional layer
    conv_layer_final = Conv2D(
        filters=num_classes,
        kernel_size=1,
        use_bias=use_bias,
        name="last_conv",
    )(x)

    # configure fully convolutional ResNet50 model
    model = training.Model(img_input, x)

    # load model weights
    if pretrained_resnet:
        model_name = "resnet50"
        # configure full file name
        file_name = model_name + 
            "_weights_tf_dim_ordering_tf_kernels_notop.h5"
        # get the file hash from TF WEIGHTS_HASHES
        file_hash = WEIGHTS_HASHES[model_name][1]
        weights_path = data_utils.get_file(
            file_name,
            BASE_WEIGHTS_PATH + file_name,
            cache_subdir="models",
            file_hash=file_hash,
        )

        model.load_weights(weights_path)

    # form final model
    model = training.Model(inputs=model.input, outputs= 
        [conv_layer_final])

    if pretrained_resnet:
        # get model with the dense layer for further FC weights 
        extraction
        resnet50_extractor = ResNet50(
            include_top=True,
            weights="imagenet",
            classes=num_classes,
        )
        # set ResNet50 FC-layer weights to final conv layer
        set_conv_weights(
            model=model,
            feature_extractor=resnet50_extractor
        )
    return model

In [ ]:
# setting FC weights to the final convolutional layer
def set_conv_weights(model, feature_extractor):
    # get pre-trained ResNet50 FC weights
    dense_layer_weights = feature_extractor.layers[-1].get_weights()
    weights_list = [
        tf.reshape(
            dense_layer_weights[0], (1, 1, *dense_layer_weights[0].shape),
        ).numpy(),
        dense_layer_weights[1],
    ]
    model.get_layer(name="last_conv").set_weights(weights_list)